<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_teaching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Generated Agent Chat: Teaching

AutoGen offers conversable agents powered by LLMs, tools, or humans, which can be used to perform tasks collectively via automated chat. This framework makes it easy to build many advanced applications of LLMs.
Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

This notebook demonstrates how AutoGen enables a user to teach AI new skills via natural agent interactions, without requiring knowledge of programming language. It is modified based on https://github.com/microsoft/FLAML/blob/evaluation/notebook/research_paper/teaching.ipynb and https://github.com/microsoft/FLAML/blob/evaluation/notebook/research_paper/teaching_recipe_reuse.ipynb.

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install "pyautogen>=0.2.3"
```

In [1]:
%pip install --quiet "pyautogen>=0.2.3"

Note: you may need to restart the kernel to use updated packages.


## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well).

The json looks like the following:
```json
[
    {
        "model": "gpt-4",
        "api_key": "<your OpenAI API key here>"
    },
    {
        "model": "gpt-4",
        "api_key": "<your Azure OpenAI API key here>",
        "base_url": "<your Azure OpenAI API base here>",
        "api_type": "azure",
        "api_version": "2023-06-01-preview"
    },
    {
        "model": "gpt-4-32k",
        "api_key": "<your Azure OpenAI API key here>",
        "base_url": "<your Azure OpenAI API base here>",
        "api_type": "azure",
        "api_version": "2023-06-01-preview"
    }
]
```

If you open this notebook in colab, you can upload your files by clicking the file icon on the left panel and then choose "upload file" icon.


In [2]:
# import autogen

# llm_config = {
#     "timeout": 600,
#     "cache_seed": 44,  # change the seed for different trials
#     "config_list": autogen.config_list_from_json(
#         "OAI_CONFIG_LIST",
#         filter_dict={"model": ["gpt-3.5-turbo-0125"]},
#     ),
#     "temperature": 0,
# }

In [3]:
import autogen

llm_config = {
    "timeout": 600,
    "cache_seed": 44,  # change the seed for different trials
    "config_list": autogen.config_list_from_json(
        "OAI_CONFIG_LIST",
        filter_dict={"model": ["ollama/mixtral"]},
    ),
    "temperature": 0,
}

## Example Task: Literature Survey

We consider a scenario where one needs to find research papers of a certain topic, categorize the application domains, and plot a bar chart of the number of papers in each domain.

### Construct Agents

We create an assistant agent to solve tasks with coding and language skills. We create a user proxy agent to describe tasks and execute the code suggested by the assistant agent.

In [4]:
# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
    is_termination_msg=lambda x: True if "TERMINATE" in x.get("content") else False,
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: True if "TERMINATE" in x.get("content") else False,
    max_consecutive_auto_reply=10,
    code_execution_config={
        "work_dir": "work_dir",
        "use_docker": False,
    },
)

### Step-by-step Requests

In [5]:
task1 = """
Find arxiv papers that show how are people studying trust calibration in AI based systems
"""

user_proxy.initiate_chat(assistant, message=task1)

user_proxy (to assistant):


Find arxiv papers that show how are people studying trust calibration in AI based systems


--------------------------------------------------------------------------------
assistant (to user_proxy):

 I cannot directly execute code or browse the web to find specific files or information for you. However, I can suggest a Python script that you can use to search for ArXiv papers related to "trust calibration in AI based systems" using the `arxiv` package. Here's the code:

```python
# filename: arxiv_search.py
import arxiv

query = "trust calibration in AI based systems"

def search(query):
    query_entry = arxiv.query(query)
    results = query_entry.search()
    return results

results = search(query)
for result in results:
    print(f"Title: {result.title}")
    print(f"Authors: {', '.join(result.author)}")
    print(f"Link: {result.url}")
    print("--------------------")
```

To use this script, you need to install the `arxiv` package first by running:

/home/twix/miniconda3/envs/genai_samples/lib/python3.10/site-packages/autogen/agentchat/conversable_agent.py:838: UserWarning: No summary_method provided or summary_method is not supported: 
  warnings.warn("No summary_method provided or summary_method is not supported: ")


ChatResult(chat_history=[{'content': '\nFind arxiv papers that show how are people studying trust calibration in AI based systems\n', 'role': 'assistant'}, {'content': ' I cannot directly execute code or browse the web to find specific files or information for you. However, I can suggest a Python script that you can use to search for ArXiv papers related to "trust calibration in AI based systems" using the `arxiv` package. Here\'s the code:\n\n```python\n# filename: arxiv_search.py\nimport arxiv\n\nquery = "trust calibration in AI based systems"\n\ndef search(query):\n    query_entry = arxiv.query(query)\n    results = query_entry.search()\n    return results\n\nresults = search(query)\nfor result in results:\n    print(f"Title: {result.title}")\n    print(f"Authors: {\', \'.join(result.author)}")\n    print(f"Link: {result.url}")\n    print("--------------------")\n```\n\nTo use this script, you need to install the `arxiv` package first by running:\n\n```sh\npip install arxiv\n```\n\n

In [6]:
task2 = "analyze the above the results to list the application domains studied by these papers "
user_proxy.initiate_chat(assistant, message=task2, clear_history=False)

user_proxy (to assistant):

analyze the above the results to list the application domains studied by these papers 

--------------------------------------------------------------------------------
assistant (to user_proxy):

 To analyze the results and extract the application domains studied in the papers, we can modify the existing Python script by adding a new variable `application_domains` to store the relevant keywords for each domain. Then, we will search for these keywords within the paper titles and print out the application domains along with the paper information. Here's the updated code:

```python
# filename: arxiv_search.py
import arxiv

query = "trust calibration in AI based systems"
application_domains = ["autonomous vehicles", "healthcare", "finance", "cybersecurity"]

def search(query):
    query_entry = arxiv.query(query)
    results = query_entry.search()
    return results

results = search(query)
for result in results:
    title = result.title
    authors = ", ".joi

ChatResult(chat_history=[{'content': '\nFind arxiv papers that show how are people studying trust calibration in AI based systems\n', 'role': 'assistant'}, {'content': ' I cannot directly execute code or browse the web to find specific files or information for you. However, I can suggest a Python script that you can use to search for ArXiv papers related to "trust calibration in AI based systems" using the `arxiv` package. Here\'s the code:\n\n```python\n# filename: arxiv_search.py\nimport arxiv\n\nquery = "trust calibration in AI based systems"\n\ndef search(query):\n    query_entry = arxiv.query(query)\n    results = query_entry.search()\n    return results\n\nresults = search(query)\nfor result in results:\n    print(f"Title: {result.title}")\n    print(f"Authors: {\', \'.join(result.author)}")\n    print(f"Link: {result.url}")\n    print("--------------------")\n```\n\nTo use this script, you need to install the `arxiv` package first by running:\n\n```sh\npip install arxiv\n```\n\n

In [7]:
task3 = """Use this data to generate a bar chart of domains and number of papers in that domain and save to a file
"""
user_proxy.initiate_chat(assistant, message=task3, clear_history=False)

user_proxy (to assistant):

Use this data to generate a bar chart of domains and number of papers in that domain and save to a file


--------------------------------------------------------------------------------
assistant (to user_proxy):

 To generate a bar chart from the data obtained using the Python script, we can use the `matplotlib` library for creating visualizations. First, let's modify the existing script to store the application domain information in a list instead of printing it directly. Then, after the loop, we will create a dictionary with the domains as keys and the number of matching papers as values. Finally, we will use `matplotlib` to generate a bar chart and save it as a PNG file. Here's the updated code:

```python
# filename: arxiv_search.py
import arxiv
import matplotlib.pyplot as plt

query = "trust calibration in AI based systems"
application_domains = ["autonomous vehicles", "healthcare", "finance", "cybersecurity"]

def search(query):
    query_entry = arx

ChatResult(chat_history=[{'content': '\nFind arxiv papers that show how are people studying trust calibration in AI based systems\n', 'role': 'assistant'}, {'content': ' I cannot directly execute code or browse the web to find specific files or information for you. However, I can suggest a Python script that you can use to search for ArXiv papers related to "trust calibration in AI based systems" using the `arxiv` package. Here\'s the code:\n\n```python\n# filename: arxiv_search.py\nimport arxiv\n\nquery = "trust calibration in AI based systems"\n\ndef search(query):\n    query_entry = arxiv.query(query)\n    results = query_entry.search()\n    return results\n\nresults = search(query)\nfor result in results:\n    print(f"Title: {result.title}")\n    print(f"Authors: {\', \'.join(result.author)}")\n    print(f"Link: {result.url}")\n    print("--------------------")\n```\n\nTo use this script, you need to install the `arxiv` package first by running:\n\n```sh\npip install arxiv\n```\n\n

In [8]:
# from IPython.display import Image

# Image(filename='work_dir/domains_bar_chart.png')

## Create Recipes

Now that the task has finished via a number of interactions. The user does not want to repeat these many steps in future. What can the user do?

A followup request can be made to create a reusable recipe.

In [9]:
task4 = """Reflect on the sequence and create a recipe containing all the steps
necessary and name for it. Suggest well-documented, generalized python function(s)
 to perform similar tasks for coding steps in future. Make sure coding steps and
 non-coding steps are never mixed in one function. In the docstr of the function(s),
 clarify what non-coding steps are needed to use the language skill of the assistant.
"""
user_proxy.initiate_chat(assistant, message=task4, clear_history=False)

user_proxy (to assistant):

Reflect on the sequence and create a recipe containing all the steps
necessary and name for it. Suggest well-documented, generalized python function(s)
 to perform similar tasks for coding steps in future. Make sure coding steps and
 non-coding steps are never mixed in one function. In the docstr of the function(s),
 clarify what non-coding steps are needed to use the language skill of the assistant.


--------------------------------------------------------------------------------
assistant (to user_proxy):

 Title: Automated ArXiv Paper Search and Visualization using Python

Recipe:

1. Install required libraries: `arxiv` and `matplotlib`.
2. Write a Python script named "arxiv\_search.py" to perform the following tasks:
   a. Define the query and application domains.
   b. Use the `arxiv` library to search for papers related to the query.
   c. Check if each paper's title or authors contain any of the specified application domains.
   d. Store the matching

ChatResult(chat_history=[{'content': '\nFind arxiv papers that show how are people studying trust calibration in AI based systems\n', 'role': 'assistant'}, {'content': ' I cannot directly execute code or browse the web to find specific files or information for you. However, I can suggest a Python script that you can use to search for ArXiv papers related to "trust calibration in AI based systems" using the `arxiv` package. Here\'s the code:\n\n```python\n# filename: arxiv_search.py\nimport arxiv\n\nquery = "trust calibration in AI based systems"\n\ndef search(query):\n    query_entry = arxiv.query(query)\n    results = query_entry.search()\n    return results\n\nresults = search(query)\nfor result in results:\n    print(f"Title: {result.title}")\n    print(f"Authors: {\', \'.join(result.author)}")\n    print(f"Link: {result.url}")\n    print("--------------------")\n```\n\nTo use this script, you need to install the `arxiv` package first by running:\n\n```sh\npip install arxiv\n```\n\n

## Reuse Recipes

The user can apply the same recipe to similar tasks in future.

### Example Application

In [10]:
# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
    is_termination_msg=lambda x: True if "TERMINATE" in x.get("content") else False,
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: True if "TERMINATE" in x.get("content") else False,
    max_consecutive_auto_reply=10,
    code_execution_config={
        "work_dir": "work_dir",
        "use_docker": False,
    },
)

task1 = '''
This recipe is available for you to reuse..

<begin recipe>
**Recipe Name:** Analyzing and Visualizing Application Domains in Arxiv Papers

**Steps:**

1. Collect relevant papers from arxiv using a search query.
2. Analyze the abstracts of the collected papers to identify application domains.
3. Count the number of papers in each application domain.
4. Generate a bar chart of the application domains and the number of papers in each domain.
5. Save the bar chart as an image file.

Here are the well-documented, generalized Python functions to perform the coding steps in the future:

```python
import requests
import feedparser
import matplotlib.pyplot as plt
from typing import List, Dict

def search_arxiv(query: str, max_results: int = 10) -> List[Dict[str, str]]:
    """
    Search arxiv for papers related to a specific query.

    :param query: The search query for arxiv papers.
    :param max_results: The maximum number of results to return. Default is 10.
    :return: A list of dictionaries containing the title, link, and summary of each paper.
    """
    base_url = "http://export.arxiv.org/api/query?"
    search_query = f"search_query=all:{query}"
    start = 0
    max_results = f"max_results={max_results}"
    url = f"{base_url}{search_query}&start={start}&{max_results}"
    response = requests.get(url)
    feed = feedparser.parse(response.content)

    papers = [{"title": entry.title, "link": entry.link, "summary": entry.summary} for entry in feed.entries]
    return papers

def generate_bar_chart(domains: Dict[str, int], output_file: str) -> None:
    """
    Generate a bar chart of application domains and the number of papers in each domain, and save it as an image file.

    :param domains: A dictionary containing application domains as keys and the number of papers as values.
    :param output_file: The name of the output image file.
    """
    fig, ax = plt.subplots()
    ax.bar(domains.keys(), domains.values())
    plt.xticks(rotation=45, ha="right")
    plt.xlabel("Application Domains")
    plt.ylabel("Number of Papers")
    plt.title("Number of Papers per Application Domain")

    plt.tight_layout()
    plt.savefig(output_file)
    plt.show()
```

**Usage:**

1. Use the `search_arxiv` function to collect relevant papers from arxiv using a search query.
2. Analyze the abstracts of the collected papers using your language skills to identify application domains and count the number of papers in each domain.
3. Use the `generate_bar_chart` function to generate a bar chart of the application domains and the number of papers in each domain, and save it as an image file.

</end recipe>


Here is a new task:
Plot a chart for application domains of GPT models
'''

user_proxy.initiate_chat(assistant, message=task1)

user_proxy (to assistant):


This recipe is available for you to reuse..

<begin recipe>
**Recipe Name:** Analyzing and Visualizing Application Domains in Arxiv Papers

**Steps:**

1. Collect relevant papers from arxiv using a search query.
2. Analyze the abstracts of the collected papers to identify application domains.
3. Count the number of papers in each application domain.
4. Generate a bar chart of the application domains and the number of papers in each domain.
5. Save the bar chart as an image file.

Here are the well-documented, generalized Python functions to perform the coding steps in the future:

```python
import requests
import feedparser
import matplotlib.pyplot as plt
from typing import List, Dict

def search_arxiv(query: str, max_results: int = 10) -> List[Dict[str, str]]:
    """
    Search arxiv for papers related to a specific query.

    :param query: The search query for arxiv papers.
    :param max_results: The maximum number of results to return. Default is 10.


ChatResult(chat_history=[{'content': '\nThis recipe is available for you to reuse..\n\n<begin recipe>\n**Recipe Name:** Analyzing and Visualizing Application Domains in Arxiv Papers\n\n**Steps:**\n\n1. Collect relevant papers from arxiv using a search query.\n2. Analyze the abstracts of the collected papers to identify application domains.\n3. Count the number of papers in each application domain.\n4. Generate a bar chart of the application domains and the number of papers in each domain.\n5. Save the bar chart as an image file.\n\nHere are the well-documented, generalized Python functions to perform the coding steps in the future:\n\n```python\nimport requests\nimport feedparser\nimport matplotlib.pyplot as plt\nfrom typing import List, Dict\n\ndef search_arxiv(query: str, max_results: int = 10) -> List[Dict[str, str]]:\n    """\n    Search arxiv for papers related to a specific query.\n\n    :param query: The search query for arxiv papers.\n    :param max_results: The maximum number 